In [1]:
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from scipy.optimize import curve_fit,Bounds
from scipy.integrate import simps
import zfit_physics as zphysics
import zfit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import mplhep
import uncertainties 
from uncertainties import ufloat
import tensorflow as tf

plt.style.use('belle2')
#%jsroot on


%run /afs/desy.de/user/j/jipatel/MT/MC_fit/Files_After_Acceptance/Signal_Background_thesis_final/Function.ipynb

2024-09-03 19:48:03.585741: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 19:48:03.620140: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 19:48:03.621334: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 19:48:04.575563: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warnin

## 200

In [3]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 200
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"],  'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"], 'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"],  'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"],  'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)


    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_200.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_200.csv'")


Running iteration 1/300...
200
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1401                 237.075  -     18   +     19       False
background_yield_0.1401             958.956  -     32   +     33       False
frac_Mbc_bkg1_0.1401               0.500036  -  0.039   +  0.039       False
frac_deltaE_bkg1_0.1401              0.4576  -   0.11   +   0.11       False
True
Iteration 1 - Signal Yield: 237.0745740019792 ± 18.59971194664039
Running iteration 2/300...
200
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5643                 230.845  -     18   +     19       False
background_yield_0.5643             965.288  -     32   +     33       False
frac_Mbc_bkg1_0.5643               0.510645  -  0.038   +  0.039       False
frac_deltaE_

## 150

In [4]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 150
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_150.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_150.csv'")


Running iteration 1/300...
150
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0756                 183.429  -     17   +     17       False
background_yield_0.0756             962.688  -     32   +     33       False
frac_Mbc_bkg1_0.0756               0.505266  -  0.039   +  0.039       False
frac_deltaE_bkg1_0.0756            0.467728  -   0.11   +   0.11       False
True
Iteration 1 - Signal Yield: 183.42940403582446 ± 17.009029994284653
Running iteration 2/300...
150
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6216                  180.44  -     17   +     17       False
background_yield_0.6216             965.638  -     32   +     33       False
frac_Mbc_bkg1_0.6216               0.511403  -  0.038   +  0.038       False
frac_delta

## 100

In [7]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 100
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_100.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_100.csv'")


Running iteration 1/300...
100
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7474                 134.333  -     15   +     16       False
background_yield_0.7474             961.666  -     32   +     33       False
frac_Mbc_bkg1_0.7474               0.504523  -  0.039   +  0.038       False
frac_deltaE_bkg1_0.7474             0.47129  -   0.11   +   0.11       False
True
Iteration 1 - Signal Yield: 134.3331315988297 ± 15.274761268064966
Running iteration 2/300...
100
996
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.149                 132.236  -     15   +     15       False
background_yield_0.149              963.78  -     32   +     33       False
frac_Mbc_bkg1_0.149               0.508724  -  0.038   +  0.038       False
frac_deltaE_bkg1

## 75

In [8]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 75
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_75.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_75.csv'")


Running iteration 1/300...
75
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6842                  110.55  -     14   +     15       False
background_yield_0.6842             960.447  -     32   +     33       False
frac_Mbc_bkg1_0.6842               0.503057  -  0.038   +  0.038       False
frac_deltaE_bkg1_0.6842             0.46039  -   0.11   +   0.11       False
True
Iteration 1 - Signal Yield: 110.5496297260261 ± 14.222733739135272
Running iteration 2/300...
75
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9586                 109.436  -     14   +     15       False
background_yield_0.9586             960.392  -     31   +     34       False
frac_Mbc_bkg1_0.9586               0.505678  -  0.038   +  0.038       False
frac_deltaE_b

## 50

In [10]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 50
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_50.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_50.csv'")


Running iteration 1/300...
50
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8174                  85.625  -     13   +     13       False
background_yield_0.8174             960.442  -     32   +     33       False
frac_Mbc_bkg1_0.8174               0.503426  -  0.038   +  0.038       False
frac_deltaE_bkg1_0.8174            0.457238  -   0.11   +   0.11       False
True
Iteration 1 - Signal Yield: 85.62501284943467 ± 13.115020237185648
Running iteration 2/300...
50
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5904                 82.5686  -     13   +     13       False
background_yield_0.5904             963.586  -     32   +     33       False
frac_Mbc_bkg1_0.5904                0.50858  -  0.039   +  0.038       False
frac_deltaE_b

## 25

In [11]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 25
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_25.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_25.csv'")


Running iteration 1/300...
25
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9748                 58.2621  -     12   +     12       False
background_yield_0.9748             962.547  -     32   +     33       False
frac_Mbc_bkg1_0.9748               0.505828  -  0.038   +  0.038       False
frac_deltaE_bkg1_0.9748            0.463521  -   0.11   +    0.1       False
True
Iteration 1 - Signal Yield: 58.26208957157969 ± 11.884763195666924
Running iteration 2/300...
25
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6739                 61.2836  -     12   +     12       False
background_yield_0.6739             959.555  -     32   +     33       False
frac_Mbc_bkg1_0.6739               0.502366  -  0.038   +  0.038       False
frac_deltaE_b

## 10

In [12]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 10
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_10.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_10.csv'")


Running iteration 1/300...
10
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2604                 41.0769  -     10   +     12       False
background_yield_0.2604             964.919  -     32   +     32       False
frac_Mbc_bkg1_0.2604                0.50893  -  0.038   +  0.038       False
frac_deltaE_bkg1_0.2604            0.469665  -   0.11   +    0.1       False
True
Iteration 1 - Signal Yield: 41.07690671644888 ± 11.02085964327914
Running iteration 2/300...
10
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.9525                 43.2885  -     10   +     12       False
background_yield_0.9525             962.745  -     32   +     32       False
frac_Mbc_bkg1_0.9525               0.506485  -  0.038   +  0.038       False
frac_deltaE_bk

## 5

In [13]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 5
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_5.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_5.csv'")


Running iteration 1/300...
5
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0094                  35.658  -     10   +     11       False
background_yield_0.0094             965.683  -     32   +     32       False
frac_Mbc_bkg1_0.0094               0.510556  -  0.038   +  0.037       False
frac_deltaE_bkg1_0.0094            0.467917  -   0.11   +    0.1       False
True
Iteration 1 - Signal Yield: 35.658019868500325 ± 10.729861316447693
Running iteration 2/300...
5
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7186                 35.2128  -     10   +     11       False
background_yield_0.7186              966.17  -     32   +     32       False
frac_Mbc_bkg1_0.7186               0.511557  -  0.038   +  0.037       False
frac_deltaE_bk

## 2

In [14]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 2
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_2.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_2.csv'")


Running iteration 1/300...
2
996
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2398                 33.1943  -     10   +     11       False
background_yield_0.2398             965.211  -     32   +     32       False
frac_Mbc_bkg1_0.2398               0.509941  -  0.038   +  0.037       False
frac_deltaE_bkg1_0.2398            0.460187  -    0.1   +   0.11       False
True
Iteration 1 - Signal Yield: 33.19433087250803 ± 10.507970794196195
Running iteration 2/300...
2
996
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.768                 32.5301  -     10   +     11       False
background_yield_0.768             965.897  -     32   +     32       False
frac_Mbc_bkg1_0.768               0.510899  -  0.038   +  0.037       False
frac_deltaE_bkg1_0.7

## 0

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_Bppip,file_bkg_mu_Bppip, 'mu', '521', 'pip', 'mumu', 4, 'rec521', 'weights', 0
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg',  mc_mu_bkg["deltaE"],  'poly', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('Bppip_q2reg4_decay_mumu_signal_yields_and_errors_0.csv', index=False)

print("Signal yields and errors saved to 'Bppip_q2reg4_decay_mumu_signal_yields_and_errors_0.csv'")


Running iteration 1/300...
0
996


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4961                 30.0203  -     10   +     11       False
background_yield_0.4961             965.943  -     32   +     33       False
frac_Mbc_bkg1_0.4961               0.511236  -  0.038   +  0.038       False
frac_deltaE_bkg1_0.4961            0.467831  -   0.11   +   0.11       False
True
Iteration 1 - Signal Yield: 30.020259364347293 ± 10.374043328146954
Running iteration 2/300...
0
996
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.437                 30.0203  -     10   +     11       False
background_yield_0.437             965.943  -     32   +     33       False
frac_Mbc_bkg1_0.437               0.511236  -  0.038   +  0.038       False
frac_deltaE_bkg1_0.437            0.467831  -   0.11